<a href="https://colab.research.google.com/github/itsamirkhon/data-auto-label/blob/main/auro_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q librosa soundfile pydub tqdm audioread

In [3]:
import os
import json
import librosa
import soundfile as sf
import numpy as np
from pathlib import Path
from pydub import AudioSegment
from pydub.silence import split_on_silence, detect_nonsilent
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки успешно импортированы")
print(f"📅 Дата: 2025-10-02 16:11:55 UTC")
print(f"👤 Пользователь: itsamirkhon")

✅ Библиотеки успешно импортированы
📅 Дата: 2025-10-02 16:11:55 UTC
👤 Пользователь: itsamirkhon


In [4]:
from google.colab import drive
drive.mount('/content/drive')
print("✅ Google Drive подключен")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive подключен


In [5]:
# ============================================================================
# КОНФИГУРАЦИЯ ПРОЕКТА
# ============================================================================

# Пути к данным
INPUT_AUDIO_PATH = "/content/drive/MyDrive/ASR_Project/raw_audio"
OUTPUT_AUDIO_PATH = "/content/drive/MyDrive/ASR_Project/processed_audio"
SEGMENTED_PATH = "/content/drive/MyDrive/ASR_Project/segmented_audio"
REPORTS_PATH = "/content/drive/MyDrive/ASR_Project/reports"

# Параметры обработки аудио
TARGET_SAMPLE_RATE = 16000  # 16 kHz - стандарт для ASR
TARGET_CHANNELS = 1         # Моно
NORMALIZE_AUDIO = True      # Нормализовать громкость
TARGET_LEVEL_DB = -20       # Целевой уровень громкости в dB

# Параметры сегментации
RUN_SEGMENTATION = True     # Включить сегментацию
MAX_DURATION = 15           # Максимальная длительность сегмента (секунды)
MIN_DURATION = 1            # Минимальная длительность сегмента (секунды)
MIN_SILENCE_LEN = 300       # Минимальная длина тишины для разделения (мс)
SILENCE_THRESH = -35        # Порог тишины в dB (-35 = довольно тихо)
SEEK_STEP = 10              # Шаг поиска тишины (мс) - меньше = точнее но медленнее

# Визуализация
SHOW_VISUALIZATIONS = True  # Показывать графики
N_SAMPLE_VISUALIZATIONS = 3 # Количество примеров для визуализации

# Создаем директории
for path in [OUTPUT_AUDIO_PATH, SEGMENTED_PATH, REPORTS_PATH]:
    os.makedirs(path, exist_ok=True)

print("="*70)
print("🎵 АВТОМАТИЧЕСКАЯ ПОДГОТОВКА АУДИО ДАННЫХ ДЛЯ ASR")
print("="*70)
print(f"📂 Входные данные:      {INPUT_AUDIO_PATH}")
print(f"🔧 Обработанные:        {OUTPUT_AUDIO_PATH}")
print(f"✂️  Сегментированные:    {SEGMENTED_PATH}")
print(f"📊 Отчеты:              {REPORTS_PATH}")
print("="*70)
print(f"⚙️  ПАРАМЕТРЫ:")
print(f"   • Sample Rate:       {TARGET_SAMPLE_RATE} Hz")
print(f"   • Channels:          {TARGET_CHANNELS} (моно)")
print(f"   • Normalize:         {NORMALIZE_AUDIO} ({TARGET_LEVEL_DB} dB)")
print(f"   • Сегментация:       {RUN_SEGMENTATION}")
print(f"   • Макс. длит. сегм:  {MAX_DURATION} сек")
print(f"   • Мин. длит. сегм:   {MIN_DURATION} сек")
print(f"   • Порог тишины:      {SILENCE_THRESH} dB")
print(f"   • Мин. длина тишины: {MIN_SILENCE_LEN} мс")
print("="*70 + "\n")

🎵 АВТОМАТИЧЕСКАЯ ПОДГОТОВКА АУДИО ДАННЫХ ДЛЯ ASR
📂 Входные данные:      /content/drive/MyDrive/ASR_Project/raw_audio
🔧 Обработанные:        /content/drive/MyDrive/ASR_Project/processed_audio
✂️  Сегментированные:    /content/drive/MyDrive/ASR_Project/segmented_audio
📊 Отчеты:              /content/drive/MyDrive/ASR_Project/reports
⚙️  ПАРАМЕТРЫ:
   • Sample Rate:       16000 Hz
   • Channels:          1 (моно)
   • Normalize:         True (-20 dB)
   • Сегментация:       True
   • Макс. длит. сегм:  15 сек
   • Мин. длит. сегм:   1 сек
   • Порог тишины:      -35 dB
   • Мин. длина тишины: 300 мс



In [6]:
class AudioDatasetAnalyzer:
    """Класс для анализа аудио датасета"""

    def __init__(self, audio_dir):
        self.audio_dir = audio_dir
        self.supported_formats = ['.wav', '.mp3', '.flac', '.ogg', '.m4a', '.aac', '.wma']
        self.audio_files = []
        self.stats = []

    def scan_files(self):
        """Сканирует директорию на наличие аудио файлов"""
        print("🔎 Сканирование аудио файлов...")

        if not os.path.exists(self.audio_dir):
            print(f"❌ Директория не существует: {self.audio_dir}")
            return []

        for root, dirs, files in os.walk(self.audio_dir):
            for file in files:
                if any(file.lower().endswith(fmt) for fmt in self.supported_formats):
                    self.audio_files.append(os.path.join(root, file))

        print(f"✅ Найдено {len(self.audio_files)} аудио файлов")
        return self.audio_files

    def analyze_file(self, file_path):
        """Анализирует один аудио файл"""
        try:
            # Получаем информацию без полной загрузки
            info = sf.info(file_path)
            duration = info.duration
            sr = info.samplerate
            channels = info.channels

            # Загружаем только первые 10 секунд для анализа громкости
            y, _ = librosa.load(file_path, sr=sr, duration=10)

            file_size = os.path.getsize(file_path) / (1024 * 1024)  # MB
            rms = np.sqrt(np.mean(y**2))
            rms_db = 20 * np.log10(rms) if rms > 0 else -100

            return {
                'file_path': file_path,
                'file_name': os.path.basename(file_path),
                'format': os.path.splitext(file_path)[1].lower(),
                'sample_rate': sr,
                'duration': duration,
                'file_size_mb': file_size,
                'channels': channels,
                'rms_energy': rms,
                'rms_db': rms_db,
                'status': 'OK'
            }
        except Exception as e:
            return {
                'file_path': file_path,
                'file_name': os.path.basename(file_path),
                'format': os.path.splitext(file_path)[1].lower() if '.' in file_path else 'unknown',
                'status': f'ERROR: {str(e)}'
            }

    def analyze_dataset(self):
        """Анализирует весь датасет"""
        if not self.audio_files:
            return pd.DataFrame()

        print(f"📊 Анализ {len(self.audio_files)} файлов...")

        for file_path in tqdm(self.audio_files, desc="Анализ"):
            stats = self.analyze_file(file_path)
            self.stats.append(stats)

        return pd.DataFrame(self.stats)

    def generate_report(self, df):
        """Генерирует отчет по датасету"""
        if df.empty:
            print("⚠️ Нет данных для отчета")
            return df

        print("\n" + "="*70)
        print("📈 ОТЧЕТ ПО ИСХОДНОМУ ДАТАСЕТУ")
        print("="*70)

        total_files = len(df)
        ok_files = len(df[df['status'] == 'OK'])
        error_files = total_files - ok_files

        print(f"\n📁 ФАЙЛЫ:")
        print(f"   • Всего:              {total_files}")
        print(f"   • Успешно обработано: {ok_files}")
        print(f"   • Ошибок:             {error_files}")

        if ok_files > 0:
            df_ok = df[df['status'] == 'OK'].copy()

            print(f"\n⏱️  ДЛИТЕЛЬНОСТЬ:")
            total_hours = df_ok['duration'].sum() / 3600
            print(f"   • Общая:   {total_hours:.2f} ч ({df_ok['duration'].sum():.1f} сек)")
            print(f"   • Средняя: {df_ok['duration'].mean():.2f} сек")
            print(f"   • Минимум: {df_ok['duration'].min():.2f} сек")
            print(f"   • Максимум: {df_ok['duration'].max():.2f} сек")

            print(f"\n🎵 ЧАСТОТА ДИСКРЕТИЗАЦИИ:")
            for sr, count in df_ok['sample_rate'].value_counts().sort_index(ascending=False).items():
                print(f"   • {int(sr):>6} Hz: {count:>3} файлов")

            print(f"\n📊 ФОРМАТЫ:")
            for fmt, count in df_ok['format'].value_counts().items():
                print(f"   • {fmt:>5}: {count:>3} файлов")

            print(f"\n🔊 КАНАЛЫ:")
            for ch, count in df_ok['channels'].value_counts().items():
                print(f"   • {int(ch)} канал(ов): {count} файлов")

            print(f"\n🎚️  ГРОМКОСТЬ:")
            print(f"   • Средний RMS: {df_ok['rms_db'].mean():.1f} dB")
            print(f"   • Минимум:     {df_ok['rms_db'].min():.1f} dB")
            print(f"   • Максимум:    {df_ok['rms_db'].max():.1f} dB")

            print(f"\n💾 РАЗМЕР:")
            print(f"   • Общий:   {df_ok['file_size_mb'].sum():.2f} MB")
            print(f"   • Средний: {df_ok['file_size_mb'].mean():.2f} MB")

        if error_files > 0:
            print(f"\n❌ ФАЙЛЫ С ОШИБКАМИ:")
            df_errors = df[df['status'] != 'OK']
            for idx, row in df_errors.iterrows():
                print(f"   • {row['file_name']}")
                print(f"     └─ {row['status']}")

        print("="*70 + "\n")

        return df

print("✅ Класс AudioDatasetAnalyzer создан")

✅ Класс AudioDatasetAnalyzer создан


In [7]:
class AudioProcessor:
    """Класс для обработки и нормализации аудио файлов"""

    def __init__(self, target_sr=16000, target_channels=1, normalize=True, target_level=-20):
        self.target_sr = target_sr
        self.target_channels = target_channels
        self.normalize = normalize
        self.target_level = target_level

    def process_file(self, input_path, output_path):
        """Обрабатывает один аудио файл"""
        try:
            # Загружаем с целевой частотой дискретизации
            y, sr = librosa.load(
                input_path,
                sr=self.target_sr,
                mono=(self.target_channels == 1)
            )

            # Нормализация громкости
            if self.normalize:
                y = self.normalize_audio(y, self.target_level)

            # Создаем директорию если не существует
            os.makedirs(os.path.dirname(output_path), exist_ok=True)

            # Сохраняем
            sf.write(output_path, y, sr, subtype='PCM_16')

            duration = len(y) / sr
            return True, None, duration

        except Exception as e:
            return False, str(e), 0

    def normalize_audio(self, audio, target_level=-20):
        """Нормализует громкость аудио к целевому уровню в dB"""
        # Рассчитываем текущий уровень
        rms = np.sqrt(np.mean(audio**2))

        if rms == 0:
            return audio

        current_db = 20 * np.log10(rms)

        # Рассчитываем требуемый gain
        gain_db = target_level - current_db
        gain = 10 ** (gain_db / 20)

        # Применяем gain
        normalized = audio * gain

        # Предотвращаем клиппинг
        max_val = np.abs(normalized).max()
        if max_val > 1.0:
            normalized = normalized / max_val * 0.99

        return normalized

    def process_dataset(self, input_dir, output_dir, file_list=None):
        """Обрабатывает весь датасет"""
        if file_list is None:
            file_list = []
            for root, dirs, files in os.walk(input_dir):
                for file in files:
                    if any(file.lower().endswith(fmt) for fmt in ['.wav', '.mp3', '.flac', '.ogg', '.m4a']):
                        file_list.append(os.path.join(root, file))

        if not file_list:
            print("⚠️ Нет файлов для обработки")
            return pd.DataFrame()

        print(f"🔧 Обработка {len(file_list)} файлов...")
        print(f"   📊 Sample Rate: {self.target_sr} Hz")
        print(f"   🔊 Channels: {self.target_channels} (моно)")
        print(f"   🎚️  Normalize: {self.normalize} (target: {self.target_level} dB)")
        print()

        results = []

        for input_path in tqdm(file_list, desc="Обработка"):
            # Создаем относительный путь
            rel_path = os.path.relpath(input_path, input_dir)
            output_path = os.path.join(output_dir, os.path.splitext(rel_path)[0] + '.wav')

            # Обрабатываем
            success, error, duration = self.process_file(input_path, output_path)

            results.append({
                'input': input_path,
                'input_name': os.path.basename(input_path),
                'output': output_path,
                'output_name': os.path.basename(output_path),
                'success': success,
                'duration': duration,
                'error': error if error else ''
            })

        return pd.DataFrame(results)

print("✅ Класс AudioProcessor создан")

✅ Класс AudioProcessor создан


In [8]:
class AudioSegmenter:
    """Класс для надежной сегментации аудио файлов"""

    def __init__(self, max_duration=15, min_duration=1, min_silence_len=300,
                 silence_thresh=-35, seek_step=10):
        self.max_duration = max_duration * 1000  # конвертируем в мс
        self.min_duration = min_duration * 1000
        self.min_silence_len = min_silence_len
        self.silence_thresh = silence_thresh
        self.seek_step = seek_step

    def force_split_segment(self, segment, max_duration_ms):
        """Принудительно разделяет длинный сегмент на части"""
        if len(segment) <= max_duration_ms:
            return [segment]

        chunks = []
        for i in range(0, len(segment), max_duration_ms):
            chunk = segment[i:i + max_duration_ms]
            if len(chunk) >= self.min_duration:
                chunks.append(chunk)

        return chunks

    def segment_by_silence(self, audio):
        """Сегментирует аудио по участкам тишины"""
        # Находим участки с речью (не тишина)
        nonsilent_ranges = detect_nonsilent(
            audio,
            min_silence_len=self.min_silence_len,
            silence_thresh=self.silence_thresh,
            seek_step=self.seek_step
        )

        if not nonsilent_ranges:
            # Если не найдено речи, возвращаем всё аудио
            return [audio]

        # Извлекаем сегменты речи
        segments = []
        for start_ms, end_ms in nonsilent_ranges:
            segment = audio[start_ms:end_ms]
            segments.append(segment)

        return segments

    def merge_and_split_segments(self, segments):
        """Объединяет короткие и разделяет длинные сегменты"""
        if not segments:
            return []

        merged = []
        current = AudioSegment.empty()

        for segment in segments:
            seg_len = len(segment)
            cur_len = len(current)

            # Если сегмент сам по себе слишком длинный
            if seg_len > self.max_duration:
                # Сохраняем накопленное
                if cur_len >= self.min_duration:
                    merged.append(current)
                    current = AudioSegment.empty()

                # Принудительно режем длинный сегмент
                chunks = self.force_split_segment(segment, self.max_duration)
                merged.extend(chunks)
                continue

            # Если добавление не превысит лимит
            if cur_len + seg_len <= self.max_duration:
                current += segment
            else:
                # Сохраняем текущий если достаточно длинный
                if cur_len >= self.min_duration:
                    merged.append(current)
                current = segment

        # Добавляем последний сегмент
        if len(current) >= self.min_duration:
            merged.append(current)

        return merged

    def validate_segments(self, segments):
        """Финальная валидация - гарантирует что все сегменты в пределах"""
        validated = []

        for segment in segments:
            seg_len_ms = len(segment)

            if seg_len_ms > self.max_duration:
                # Принудительно режем
                chunks = self.force_split_segment(segment, self.max_duration)
                validated.extend(chunks)
            elif seg_len_ms >= self.min_duration:
                validated.append(segment)

        return validated

    def segment_file(self, input_path, output_dir):
        """Сегментирует один аудио файл"""
        try:
            # Загружаем аудио
            audio = AudioSegment.from_file(input_path)
            duration_sec = len(audio) / 1000
            base_name = os.path.splitext(os.path.basename(input_path))[0]

            print(f"📼 {base_name} ({duration_sec:.1f} сек)")

            # Если файл достаточно короткий - копируем
            if duration_sec <= (self.max_duration / 1000):
                output_path = os.path.join(output_dir, f"{base_name}.wav")
                audio.export(output_path, format="wav")
                print(f"   ✅ Файл короткий, сохранен без изменений")

                return [{
                    'path': output_path,
                    'filename': f"{base_name}.wav",
                    'duration': duration_sec,
                    'method': 'copy'
                }], True

            # Файл длинный - сегментируем
            print(f"   ✂️  Файл длинный, выполняю сегментацию...")

            # Шаг 1: Разделяем по тишине
            segments = self.segment_by_silence(audio)
            print(f"   🔍 Найдено {len(segments)} сегментов по тишине")

            # Шаг 2: Объединяем короткие и разделяем длинные
            segments = self.merge_and_split_segments(segments)
            print(f"   🔄 После объединения/разделения: {len(segments)} сегментов")

            # Шаг 3: Финальная валидация
            segments = self.validate_segments(segments)
            print(f"   ✓ После валидации: {len(segments)} сегментов")

            # Сохраняем сегменты
            output_paths = []
            for i, segment in enumerate(segments):
                seg_duration = len(segment) / 1000
                output_path = os.path.join(output_dir, f"{base_name}_seg{i:03d}.wav")
                segment.export(output_path, format="wav")

                output_paths.append({
                    'path': output_path,
                    'filename': f"{base_name}_seg{i:03d}.wav",
                    'duration': seg_duration,
                    'method': 'segmented'
                })

            # Статистика
            durations = [s['duration'] for s in output_paths]
            avg_dur = np.mean(durations)
            max_dur = max(durations)
            min_dur = min(durations)

            print(f"   ✅ Создано {len(output_paths)} сегментов:")
            print(f"      • Средняя длительность: {avg_dur:.1f}с")
            print(f"      • Диапазон: {min_dur:.1f}с - {max_dur:.1f}с")

            # Предупреждение если есть слишком длинные
            if max_dur > (self.max_duration / 1000) + 0.5:
                print(f"      ⚠️  ВНИМАНИЕ: Максимальный сегмент {max_dur:.1f}с превышает лимит!")
                return output_paths, False

            return output_paths, True

        except Exception as e:
            print(f"   ❌ Ошибка: {str(e)}")
            import traceback
            traceback.print_exc()
            return [], False

    def segment_dataset(self, input_dir, output_dir):
        """Сегментирует весь датасет"""
        # Находим все WAV файлы
        audio_files = []
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith('.wav'):
                    audio_files.append(os.path.join(root, file))

        if not audio_files:
            print("⚠️ Не найдено WAV файлов для сегментации")
            return [], {}

        print(f"\n{'='*70}")
        print(f"✂️  СЕГМЕНТАЦИЯ ДАТАСЕТА")
        print(f"{'='*70}")
        print(f"📁 Входная директория:  {input_dir}")
        print(f"📁 Выходная директория: {output_dir}")
        print(f"📊 Найдено файлов:      {len(audio_files)}")
        print(f"⚙️  Параметры:")
        print(f"   • Макс. длительность: {self.max_duration / 1000:.1f}с")
        print(f"   • Мин. длительность:  {self.min_duration / 1000:.1f}с")
        print(f"   • Порог тишины:       {self.silence_thresh} dB")
        print(f"   • Мин. длина тишины:  {self.min_silence_len} мс")
        print(f"{'='*70}\n")

        all_segments = []
        stats = {
            'total_input_files': len(audio_files),
            'total_output_segments': 0,
            'copied_files': 0,
            'segmented_files': 0,
            'errors': 0,
            'validation_warnings': 0
        }

        for idx, file_path in enumerate(audio_files, 1):
            print(f"[{idx}/{len(audio_files)}]")

            segments, validated = self.segment_file(file_path, output_dir)

            if segments:
                all_segments.extend(segments)
                stats['total_output_segments'] += len(segments)

                if len(segments) == 1 and segments[0]['method'] == 'copy':
                    stats['copied_files'] += 1
                else:
                    stats['segmented_files'] += 1

                if not validated:
                    stats['validation_warnings'] += 1
            else:
                stats['errors'] += 1

            print()

        # Финальная статистика
        if all_segments:
            durations = [s['duration'] for s in all_segments]
            stats['avg_duration'] = float(np.mean(durations))
            stats['max_duration'] = float(max(durations))
            stats['min_duration'] = float(min(durations))
            stats['total_duration'] = float(sum(durations))

        print(f"{'='*70}")
        print(f"📈 РЕЗУЛЬТАТЫ СЕГМЕНТАЦИИ")
        print(f"{'='*70}")
        print(f"📥 Входных файлов:            {stats['total_input_files']}")
        print(f"📤 Выходных сегментов:        {stats['total_output_segments']}")
        print(f"📋 Скопировано без изменений: {stats['copied_files']}")
        print(f"✂️  Сегментировано:            {stats['segmented_files']}")
        print(f"❌ Ошибок:                    {stats['errors']}")

        if stats.get('validation_warnings', 0) > 0:
            print(f"⚠️  Предупреждений валидации:  {stats['validation_warnings']}")

        if all_segments:
            print(f"\n📊 СТАТИСТИКА ДЛИТЕЛЬНОСТИ:")
            print(f"   • Всего аудио:  {stats['total_duration'] / 3600:.2f} ч")
            print(f"   • Средняя:      {stats['avg_duration']:.1f} сек")
            print(f"   • Минимальная:  {stats['min_duration']:.1f} сек")
            print(f"   • Максимальная: {stats['max_duration']:.1f} сек")

            # Проверка диапазона
            max_allowed = self.max_duration / 1000
            if stats['max_duration'] > max_allowed + 0.5:
                print(f"\n⚠️  ВНИМАНИЕ: Максимальная длительность ({stats['max_duration']:.1f}с)")
                print(f"    превышает установленный лимит ({max_allowed:.1f}с)")
                print(f"    Рекомендуется:")
                print(f"    • Уменьшить SILENCE_THRESH (например, до -30 dB)")
                print(f"    • Уменьшить MIN_SILENCE_LEN (например, до 200 мс)")
            else:
                print(f"\n✅ Все сегменты в пределах лимита ({max_allowed:.1f}с)")

        print(f"{'='*70}\n")

        return all_segments, stats

print("✅ Класс AudioSegmenter создан")

✅ Класс AudioSegmenter создан


In [9]:
def visualize_audio_samples(audio_path, n_samples=3, title="Примеры аудио"):
    """Визуализирует случайные образцы аудио"""
    import random

    audio_files = []
    for root, dirs, files in os.walk(audio_path):
        for file in files:
            if file.lower().endswith('.wav'):
                audio_files.append(os.path.join(root, file))

    if not audio_files:
        print(f"⚠️ Не найдено WAV файлов в {audio_path}")
        return

    samples = random.sample(audio_files, min(n_samples, len(audio_files)))

    fig, axes = plt.subplots(n_samples, 2, figsize=(16, 4 * n_samples))
    if n_samples == 1:
        axes = axes.reshape(1, -1)

    fig.suptitle(title, fontsize=16, fontweight='bold')

    for i, file_path in enumerate(samples):
        try:
            y, sr = librosa.load(file_path, sr=None)
            duration = len(y) / sr

            # Waveform
            time_axis = np.linspace(0, duration, len(y))
            axes[i, 0].plot(time_axis, y, linewidth=0.5, color='#2E86AB')
            axes[i, 0].set_title(f'{os.path.basename(file_path)}\nДлительность: {duration:.2f}с | SR: {sr}Hz',
                                fontsize=10)
            axes[i, 0].set_xlabel('Время (сек)')
            axes[i, 0].set_ylabel('Амплитуда')
            axes[i, 0].grid(True, alpha=0.3)
            axes[i, 0].set_ylim(-1, 1)

            # Spectrogram
            D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
            img = axes[i, 1].imshow(D, aspect='auto', origin='lower',
                                   cmap='viridis', extent=[0, duration, 0, sr/2000])
            axes[i, 1].set_title('Спектрограмма', fontsize=10)
            axes[i, 1].set_xlabel('Время (сек)')
            axes[i, 1].set_ylabel('Частота (kHz)')
            plt.colorbar(img, ax=axes[i, 1], format='%+2.0f dB', label='Амплитуда (dB)')

        except Exception as e:
            axes[i, 0].text(0.5, 0.5, f'Ошибка: {str(e)}',
                          ha='center', va='center', transform=axes[i, 0].transAxes)
            axes[i, 1].text(0.5, 0.5, f'Ошибка: {str(e)}',
                          ha='center', va='center', transform=axes[i, 1].transAxes)

    plt.tight_layout()
    plt.show()

    print(f"📊 Визуализировано {len(samples)} из {len(audio_files)} файлов")

def plot_duration_distribution(segments_df, title="Распределение длительности сегментов"):
    """Строит гистограмму распределения длительности"""
    if segments_df.empty or 'duration' not in segments_df.columns:
        print("⚠️ Нет данных для построения распределения")
        return

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

    durations = segments_df['duration']

    # Гистограмма
    ax1.hist(durations, bins=30, color='#06A77D', alpha=0.7, edgecolor='black')
    ax1.axvline(durations.mean(), color='red', linestyle='--', linewidth=2, label=f'Среднее: {durations.mean():.1f}с')
    ax1.axvline(durations.median(), color='orange', linestyle='--', linewidth=2, label=f'Медиана: {durations.median():.1f}с')
    ax1.set_xlabel('Длительность (секунды)', fontsize=12)
    ax1.set_ylabel('Количество сегментов', fontsize=12)
    ax1.set_title('Гистограмма длительности', fontsize=14, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # Box plot
    ax2.boxplot(durations, vert=True, patch_artist=True,
               boxprops=dict(facecolor='#06A77D', alpha=0.7),
               medianprops=dict(color='red', linewidth=2))
    ax2.set_ylabel('Длительность (секунды)', fontsize=12)
    ax2.set_title('Box Plot длительности', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')

    plt.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

    print(f"📊 Статистика:")
    print(f"   • Количество сегментов: {len(durations)}")
    print(f"   • Среднее:     {durations.mean():.2f}с")
    print(f"   • Медиана:     {durations.median():.2f}с")
    print(f"   • Стд. откл.:  {durations.std():.2f}с")
    print(f"   • Мин:         {durations.min():.2f}с")
    print(f"   • Макс:        {durations.max():.2f}с")

print("✅ Функции визуализации созданы")

✅ Функции визуализации созданы


In [11]:
# ============================================================================
# АВТОМАТИЧЕСКИЙ ПАЙПЛАЙН ПОДГОТОВКИ АУДИО
# ============================================================================

def convert_to_serializable(obj):
    """Конвертирует numpy типы в стандартные Python типы для JSON"""
    if isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_to_serializable(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_serializable(item) for item in obj]
    return obj

print("\n" + "🎬 "*30)
print("ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА ОБРАБОТКИ АУДИО")
print("🎬 "*30 + "\n")

# Переменные для хранения результатов
df_analysis = None
df_processing = None
df_segments = None
processing_stats = None
segmentation_stats = None

# ============================================================================
# ШАГ 1: АНАЛИЗ ИСХОДНЫХ ДАННЫХ
# ============================================================================
print("="*70)
print("📊 ШАГ 1/3: Анализ исходных аудио файлов")
print("="*70 + "\n")

analyzer = AudioDatasetAnalyzer(INPUT_AUDIO_PATH)
audio_files = analyzer.scan_files()

if len(audio_files) == 0:
    print("❌ КРИТИЧЕСКАЯ ОШИБКА: Аудио файлы не найдены!")
    print(f"   Проверьте путь: {INPUT_AUDIO_PATH}")
    print("\n💡 Убедитесь что:")
    print("   1. Google Drive подключен")
    print("   2. Путь INPUT_AUDIO_PATH указан правильно")
    print("   3. В папке есть аудио файлы (.wav, .mp3, .flac и т.д.)")

else:
    df_analysis = analyzer.analyze_dataset()
    df_analysis = analyzer.generate_report(df_analysis)

    # Сохраняем отчет
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_file = os.path.join(REPORTS_PATH, f'01_analysis_{timestamp}.csv')
    df_analysis.to_csv(report_file, index=False)
    print(f"💾 Отчет сохранен: {report_file}")

    # ========================================================================
    # ШАГ 2: ОБРАБОТКА И НОРМАЛИЗАЦИЯ
    # ========================================================================
    print("\n" + "="*70)
    print("🔧 ШАГ 2/3: Обработка и нормализация аудио")
    print("="*70 + "\n")

    processor = AudioProcessor(
        target_sr=TARGET_SAMPLE_RATE,
        target_channels=TARGET_CHANNELS,
        normalize=NORMALIZE_AUDIO,
        target_level=TARGET_LEVEL_DB
    )

    df_processing = processor.process_dataset(INPUT_AUDIO_PATH, OUTPUT_AUDIO_PATH, audio_files)

    # Статистика обработки (с явным приведением типов)
    success_count = int(df_processing['success'].sum())
    error_count = int(len(df_processing) - success_count)
    total_duration = float(df_processing[df_processing['success']]['duration'].sum())

    print(f"\n{'='*70}")
    print(f"📊 РЕЗУЛЬТАТЫ ОБРАБОТКИ")
    print(f"{'='*70}")
    print(f"✅ Успешно обработано: {success_count} файлов")
    print(f"❌ Ошибок:             {error_count} файлов")
    print(f"⏱️  Общая длительность: {total_duration / 3600:.2f} часов")

    processing_stats = {
        'total_files': int(len(df_processing)),
        'success': success_count,
        'errors': error_count,
        'total_duration_hours': float(total_duration / 3600)
    }

    if error_count > 0:
        print(f"\n❌ ФАЙЛЫ С ОШИБКАМИ:")
        error_df = df_processing[df_processing['success'] == False]
        for idx, row in error_df.head(10).iterrows():
            print(f"   • {row['input_name']}")
            print(f"     └─ {row['error']}")
        if error_count > 10:
            print(f"   ... и ещё {error_count - 10} файлов")

    print(f"{'='*70}")

    # Сохраняем результаты
    processing_file = os.path.join(REPORTS_PATH, f'02_processing_{timestamp}.csv')
    df_processing.to_csv(processing_file, index=False)
    print(f"\n💾 Результаты обработки: {processing_file}")

    # ========================================================================
    # ШАГ 3: СЕГМЕНТАЦИЯ
    # ========================================================================
    if RUN_SEGMENTATION and success_count > 0:
        print("\n" + "="*70)
        print("✂️  ШАГ 3/3: Сегментация аудио")
        print("="*70 + "\n")

        segmenter = AudioSegmenter(
            max_duration=MAX_DURATION,
            min_duration=MIN_DURATION,
            min_silence_len=MIN_SILENCE_LEN,
            silence_thresh=SILENCE_THRESH,
            seek_step=SEEK_STEP
        )

        segments, segmentation_stats = segmenter.segment_dataset(OUTPUT_AUDIO_PATH, SEGMENTED_PATH)

        # Конвертируем статистику в сериализуемый формат
        segmentation_stats = convert_to_serializable(segmentation_stats)

        # Создаем DataFrame с сегментами
        if segments:
            df_segments = pd.DataFrame(segments)
            segments_file = os.path.join(REPORTS_PATH, f'03_segments_{timestamp}.csv')
            df_segments.to_csv(segments_file, index=False)
            print(f"💾 Информация о сегментах: {segments_file}")

        # Сохраняем статистику
        stats_file = os.path.join(REPORTS_PATH, f'03_segmentation_stats_{timestamp}.json')
        with open(stats_file, 'w', encoding='utf-8') as f:
            json.dump(segmentation_stats, f, indent=2, ensure_ascii=False)
        print(f"💾 Статистика сегментации: {stats_file}")

    elif not RUN_SEGMENTATION:
        print("\n⏭️  ШАГ 3/3: Сегментация пропущена (RUN_SEGMENTATION = False)")

    # ========================================================================
    # ФИНАЛЬНЫЙ ОТЧЕТ
    # ========================================================================
    print("\n" + "="*70)
    print("📋 ГЕНЕРАЦИЯ ФИНАЛЬНОГО ОТЧЕТА")
    print("="*70 + "\n")

    # Подготавливаем данные для анализа с явным приведением типов
    valid_files_count = int(len(df_analysis[df_analysis['status'] == 'OK'])) if df_analysis is not None else 0
    total_input_duration = float(df_analysis[df_analysis['status'] == 'OK']['duration'].sum()) if df_analysis is not None else 0.0

    final_report = {
        'metadata': {
            'project': 'ASR Audio Preparation Pipeline',
            'date': '2025-10-02 16:11:55 UTC',
            'user': 'itsamirkhon',
            'timestamp': timestamp
        },
        'paths': {
            'input': INPUT_AUDIO_PATH,
            'processed': OUTPUT_AUDIO_PATH,
            'segmented': SEGMENTED_PATH if RUN_SEGMENTATION else None,
            'reports': REPORTS_PATH
        },
        'settings': {
            'target_sample_rate': int(TARGET_SAMPLE_RATE),
            'target_channels': int(TARGET_CHANNELS),
            'normalize_audio': bool(NORMALIZE_AUDIO),
            'target_level_db': float(TARGET_LEVEL_DB),
            'segmentation_enabled': bool(RUN_SEGMENTATION),
            'max_segment_duration': float(MAX_DURATION),
            'min_segment_duration': float(MIN_DURATION),
            'silence_threshold': float(SILENCE_THRESH),
            'min_silence_length': int(MIN_SILENCE_LEN)
        },
        'statistics': {
            'step1_analysis': {
                'input_files': int(len(audio_files)),
                'valid_files': valid_files_count,
                'total_duration_hours': float(total_input_duration / 3600)
            },
            'step2_processing': convert_to_serializable(processing_stats) if processing_stats else {},
            'step3_segmentation': convert_to_serializable(segmentation_stats) if segmentation_stats else {}
        }
    }

    # Финальная конвертация всего отчета
    final_report = convert_to_serializable(final_report)

    final_report_file = os.path.join(REPORTS_PATH, f'00_FINAL_REPORT_{timestamp}.json')
    with open(final_report_file, 'w', encoding='utf-8') as f:
        json.dump(final_report, f, indent=2, ensure_ascii=False)

    print("✅ Финальный отчет создан")
    print(f"📄 {final_report_file}")

    print("\n" + "🎉 "*30)
    print("ВСЕ ЭТАПЫ УСПЕШНО ЗАВЕРШЕНЫ!")
    print("🎉 "*30 + "\n")

    print("📁 РЕЗУЛЬТАТЫ:")
    print(f"   • Исходные файлы:        {len(audio_files)}")
    print(f"   • Обработанные файлы:    {success_count}")
    if RUN_SEGMENTATION and segmentation_stats:
        print(f"   • Сегментированные:      {segmentation_stats.get('total_output_segments', 0)}")
    print(f"\n📂 ДИРЕКТОРИИ:")
    print(f"   • Обработанные:    {OUTPUT_AUDIO_PATH}")
    if RUN_SEGMENTATION:
        print(f"   • Сегментированные: {SEGMENTED_PATH}")
    print(f"   • Отчеты:          {REPORTS_PATH}")

    print(f"\n📊 ОТЧЕТЫ:")
    print(f"   • Анализ:          01_analysis_{timestamp}.csv")
    print(f"   • Обработка:       02_processing_{timestamp}.csv")
    if RUN_SEGMENTATION:
        print(f"   • Сегменты:        03_segments_{timestamp}.csv")
        print(f"   • Статистика:      03_segmentation_stats_{timestamp}.json")
    print(f"   • Финальный отчет: 00_FINAL_REPORT_{timestamp}.json")

    print("\n" + "="*70)
    print("🔜 СЛЕДУЮЩИЕ ШАГИ:")
    print("="*70)
    print("1. ✅ Проверьте обработанные файлы")
    print("2. ✅ Изучите отчеты в папке reports/")
    if RUN_SEGMENTATION:
        print("3. ✅ Проверьте сегментированные файлы")
        print("4. 🔜 Готово к Этапу 2: Транскрибирование")
    else:
        print("3. 🔜 Готово к Этапу 2: Транскрибирование")
    print("="*70 + "\n")


🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 
ЗАПУСК ПОЛНОГО ПАЙПЛАЙНА ОБРАБОТКИ АУДИО
🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 🎬 

📊 ШАГ 1/3: Анализ исходных аудио файлов

🔎 Сканирование аудио файлов...
✅ Найдено 24 аудио файлов
📊 Анализ 24 файлов...


Анализ:   0%|          | 0/24 [00:00<?, ?it/s]


📈 ОТЧЕТ ПО ИСХОДНОМУ ДАТАСЕТУ

📁 ФАЙЛЫ:
   • Всего:              24
   • Успешно обработано: 24
   • Ошибок:             0

⏱️  ДЛИТЕЛЬНОСТЬ:
   • Общая:   23.15 ч (83326.7 сек)
   • Средняя: 3471.95 сек
   • Минимум: 3328.95 сек
   • Максимум: 3536.45 сек

🎵 ЧАСТОТА ДИСКРЕТИЗАЦИИ:
   •  44100 Hz:  24 файлов

📊 ФОРМАТЫ:
   •  .mp3:  24 файлов

🔊 КАНАЛЫ:
   • 2 канал(ов): 24 файлов

🎚️  ГРОМКОСТЬ:
   • Средний RMS: -11.5 dB
   • Минимум:     -11.5 dB
   • Максимум:    -11.4 dB

💾 РАЗМЕР:
   • Общий:   1881.23 MB
   • Средний: 78.38 MB

💾 Отчет сохранен: /content/drive/MyDrive/ASR_Project/reports/01_analysis_20251002_161758.csv

🔧 ШАГ 2/3: Обработка и нормализация аудио

🔧 Обработка 24 файлов...
   📊 Sample Rate: 16000 Hz
   🔊 Channels: 1 (моно)
   🎚️  Normalize: True (target: -20 dB)



Обработка:   0%|          | 0/24 [00:00<?, ?it/s]


📊 РЕЗУЛЬТАТЫ ОБРАБОТКИ
✅ Успешно обработано: 24 файлов
❌ Ошибок:             0 файлов
⏱️  Общая длительность: 23.15 часов

💾 Результаты обработки: /content/drive/MyDrive/ASR_Project/reports/02_processing_20251002_161758.csv

✂️  ШАГ 3/3: Сегментация аудио


✂️  СЕГМЕНТАЦИЯ ДАТАСЕТА
📁 Входная директория:  /content/drive/MyDrive/ASR_Project/processed_audio
📁 Выходная директория: /content/drive/MyDrive/ASR_Project/segmented_audio
📊 Найдено файлов:      22
⚙️  Параметры:
   • Макс. длительность: 15.0с
   • Мин. длительность:  1.0с
   • Порог тишины:       -35 dB
   • Мин. длина тишины:  300 мс

[1/22]
📼 Copy of Copy of PAEMI SUBH 09 10 13 (3469.6 сек)
   ✂️  Файл длинный, выполняю сегментацию...
   🔍 Найдено 60 сегментов по тишине
   🔄 После объединения/разделения: 253 сегментов
   ✓ После валидации: 253 сегментов
   ✅ Создано 253 сегментов:
      • Средняя длительность: 13.6с
      • Диапазон: 1.1с - 15.0с

[2/22]
📼 Copy of Copy of PAEMI SUBH 03 10 13 (3525.8 сек)
   ✂️  Файл длинный, вы